In [2]:
import pandas as pd
from sklearn.linear_model import BayesianRidge
from sklearn.model_selection import train_test_split

In [27]:
from sklearn.metrics import mean_squared_error

df = pd.read_csv("Economic_data.csv")

relevant_columns = ['Year', 'Country', 'Inflation_Rate', 'GDP_Per_Capita', 'GDP_Growth_Rate', 'Unemployment_Rate']

df = df[relevant_columns]

def fill_na_with_country_mean(df, column):
    return df[column].fillna(df.groupby('Country')[column].transform('mean'))

df['Inflation_Rate'] = fill_na_with_country_mean(df, 'Inflation_Rate')
df['GDP_Per_Capita'] = fill_na_with_country_mean(df, 'GDP_Per_Capita')
df['GDP_Growth_Rate'] = fill_na_with_country_mean(df, 'GDP_Growth_Rate')

df_available = df[df['Year'] >= 1991]
df_missing = df[df['Year'] < 1991]

df_available['Unemployment_Rate'] = fill_na_with_country_mean(df_available, 'Unemployment_Rate')
df_available = df_available.dropna()

X = df_available[['Inflation_Rate', 'GDP_Per_Capita', 'GDP_Growth_Rate']]
y = df_available['Unemployment_Rate']

model = BayesianRidge()
model.fit(X,y)

df_missing = df_missing.dropna(subset=['Inflation_Rate'])
X_missing = df_missing[['Inflation_Rate', 'GDP_Per_Capita', 'GDP_Growth_Rate']]
df_missing['Unemployment_Rate'] = model.predict(X_missing)

df_combined = pd.concat([df_available, df_missing]).sort_values(by='Year')
df_combined.to_csv('Testing.csv', index=False)

C:\Users\Tanay\AppData\Local\Temp\ipykernel_15764\4093805967.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_available['Unemployment_Rate'] = fill_na_with_country_mean(df_available, 'Unemployment_Rate')


In [31]:
cost_of_living_df = pd.read_csv('Cost of Living.csv')
econ_data = pd.read_csv('Testing.csv')

econ_data['Country'] = econ_data['Country'].replace("Czechia", "Czech Republic")
econ_data['Country'] = econ_data['Country'].replace("Hong Kong SAR, China", "Hong Kong")
econ_data['Country'] = econ_data['Country'].replace("Korea, Rep.", "South Korea")
econ_data['Country'] = econ_data['Country'].replace("Turkiye", "Turkey")
econ_data['Country'] = econ_data['Country'].replace("Russian Federation", "Russia")
econ_data['Country'] = econ_data['Country'].replace("Slovak Republic", "Slovakia")
econ_data['Country'] = econ_data['Country'].replace("Macao SAR, China", "Taiwan")
econ_data['Country'] = econ_data['Country'].replace("Egypt, Arab Rep.", "Egypt")
econ_data['Country'] = econ_data['Country'].replace("Iran, Islamic Rep.", "Iran")
econ_data['Country'] = econ_data['Country'].replace("Viet Nam", "Vietnam")
econ_data['Country'] = econ_data['Country'].replace("Venezuela, RB", "Venezuela")
econ_data['Country'] = econ_data['Country'].replace("Bosnia and Herzegovina", "Bosnia And Herzegovina")
econ_data['Country'] = econ_data['Country'].replace("Syrian Arab Republic", "Syria")


merged_df = pd.merge(econ_data, cost_of_living_df, on=['Year', 'Country'], how='left')

merged_df.to_csv('Testing_2.csv', index=False)

In [35]:
testing_2_df = pd.read_csv('testing_2.csv')
countries_with_data_in_2012 = cost_of_living_df[cost_of_living_df['Year'] == 2012]['Country'].unique()
filtered_df = testing_2_df[testing_2_df['Country'].isin(countries_with_data_in_2012)]
filtered_df.to_csv('testing_3.csv', index=False)

### Main imputation

In [36]:
from sklearn.ensemble import RandomForestRegressor
data = pd.read_csv('testing_3.csv')

features = ['Inflation_Rate', 'GDP_Per_Capita', 'GDP_Growth_Rate', 'Unemployment_Rate']
targets = ['Cost of Living Index', 'Rent Index', 'Cost of Living Plus Rent Index', 
           'Groceries Index', 'Restaurant Price Index', 'Local Purchasing Power Index']

train_data = data[data['Year'] >= 2012].dropna(subset=targets)
test_data = data[data['Year'] < 2012]

for target in targets:
    X_train = train_data[features]
    y_train = train_data[target]
    
    model = RandomForestRegressor(n_estimators=100, random_state=42)

    model.fit(X_train, y_train)

    X_test = test_data[features]
    test_data[target] = model.predict(X_test)

test_data.to_csv('imputed_data_1970_2011.csv', index=False)

C:\Users\Tanay\AppData\Local\Temp\ipykernel_15764\260885079.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data[target] = model.predict(X_test)
C:\Users\Tanay\AppData\Local\Temp\ipykernel_15764\260885079.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  test_data[target] = model.predict(X_test)
C:\Users\Tanay\AppData\Local\Temp\ipykernel_15764\260885079.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = v